<!-- filepath: c:\Users\Artem Khakimov\Desktop\MIPT\mlops\pixxbind\demo_pixxbind.ipynb -->

# Демонстрация pixxbind (C++ аугментации + pybind11)

Wheel можно поставить в среду с совпадающим ABI:
- Windows: `dist/pixxbind-0.1.0-cp314-cp314-win_amd64.whl`
- Linux (python:3.10-slim): `dist/pixxbind-0.1.0-cp310-cp310-linux_x86_64.whl`

В ноутбуке ниже используем локальный Windows wheel.

In [3]:
# Установка совместимого wheel и опорных библиотек (opencv). Подберите whl по текущей версии Python.
import sys, subprocess, pathlib, re

tag = f"cp{sys.version_info.major}{sys.version_info.minor}"
dist_dir = pathlib.Path('dist')
if not dist_dir.exists():
    raise FileNotFoundError("Каталог dist/ не найден. Соберите пакетом: python -m build")

wheels = sorted(dist_dir.glob('pixxbind-*.whl'))
compatible = [w for w in wheels if tag in w.name]
if not compatible:
    raise FileNotFoundError(f"Не найден wheel для {tag}. Соберите его этой версией Python: python -m build")

wheel_path = compatible[0]
print(f"Используем wheel: {wheel_path}")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--force-reinstall', str(wheel_path)])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'opencv-python-headless'])
print('Установка завершена')

FileNotFoundError: Не найден wheel для cp311. Соберите его этой версией Python: python -m build

## Проверка импорта и функций
Импортируем модуль, смотрим доступные функции.

In [ ]:
import pixxbind as px
import inspect

print("pixxbind импортирован, функции:", px.__all__)
# Покажем docstring первой функции
print("to_gray doc:", inspect.getdoc(px.to_gray).split('\n')[0])

## Подготовка примеров (NumPy uint8)
Создаем маленькие RGB/градиентные изображения для наглядности.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Простая шахматка и градиент для демонстраций
rgb = np.array([
    [[30, 60, 90], [120, 150, 180], [210, 240, 30]],
    [[15, 45, 75], [105, 135, 165], [195, 225, 15]],
    [[0, 30, 60], [90, 120, 150], [180, 210, 0]],
], dtype=np.uint8)

gray = np.linspace(0, 255, 9, dtype=np.uint8).reshape(3, 3, 1)

print("rgb shape", rgb.shape, "gray shape", gray.shape)

## to_gray
Конвертация RGB в градации серого.

In [ ]:
gray_from_rgb = px.to_gray(rgb)
print("gray_from_rgb shape", gray_from_rgb.shape)

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(rgb)
axes[0].set_title('RGB')
axes[0].axis('off')
axes[1].imshow(gray_from_rgb[..., 0], cmap='gray', vmin=0, vmax=255)
axes[1].set_title('to_gray')
axes[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Эталон с OpenCV: cvtColor BGR->GRAY (наш rgb трактуем как RGB, поэтому конвертируем через COLOR_RGB2GRAY)
import cv2
cv_gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
cv_gray = cv_gray[..., None]
print("Макс. отличие с pixxbind:", np.abs(cv_gray.astype(int) - gray_from_rgb.astype(int)).max())

## Яркость и контраст
Демонстрация `brightness` и `contrast`.

In [ ]:
bright = px.brightness(rgb, 40)
contr = px.contrast(rgb, 1.5)

fig, axes = plt.subplots(1, 3, figsize=(9, 3))
for ax, img, title in zip(axes, [rgb, bright, contr], ['orig', 'brightness +40', 'contrast x1.5']):
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Эталон с OpenCV: brightness/contrast через convertScaleAbs
cv_bright = cv2.convertScaleAbs(rgb, alpha=1.0, beta=40)  # beta = delta
cv_contr = cv2.convertScaleAbs(rgb, alpha=1.5, beta=0)
print("Макс. отличие bright:", np.abs(cv_bright.astype(int) - bright.astype(int)).max())
print("Макс. отличие contrast:", np.abs(cv_contr.astype(int) - contr.astype(int)).max())

## add_noise с фиксированным seed
Детерминированный шум для воспроизводимости.

In [ ]:
noisy = px.add_noise(rgb, amp=25, seed=123)

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(rgb)
axes[0].set_title('orig')
axes[0].axis('off')
axes[1].imshow(noisy)
axes[1].set_title('add_noise amp=25')
axes[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Эталонный шум через OpenCV randu (равномерный), тот же seed
cv2.setRNGSeed(123)
noise = np.zeros_like(rgb, dtype=np.int16)
cv2.randu(noise, -25, 25)  # шум в диапазоне [-25,25]
ref_noisy = np.clip(rgb.astype(np.int16) + noise, 0, 255).astype(np.uint8)
print("Макс. отличие с pixxbind add_noise:", np.abs(ref_noisy.astype(int) - noisy.astype(int)).max())

## random_crop с seed
Кадрируем и проверяем размеры.

In [ ]:
crop = px.random_crop(rgb, out_h=2, out_w=2, seed=42)
print("crop shape", crop.shape)

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(rgb)
axes[0].set_title('orig')
axes[0].axis('off')
axes[1].imshow(crop)
axes[1].set_title('random_crop 2x2')
axes[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Эталонный random crop через NumPy с тем же seed
rs = np.random.default_rng(42)
y0 = rs.integers(0, rgb.shape[0] - 2 + 1)
x0 = rs.integers(0, rgb.shape[1] - 2 + 1)
ref_crop = rgb[y0:y0+2, x0:x0+2, :]
print("Макс. отличие с pixxbind crop:", np.abs(ref_crop.astype(int) - crop.astype(int)).max())

## resize_pad
Сохранение пропорций и паддинг.

In [ ]:
rp_keep = px.resize_pad(rgb, out_h=5, out_w=8, keep_aspect=True, pad=5, bilinear=False)
rp_stretch = px.resize_pad(rgb, out_h=5, out_w=8, keep_aspect=False, pad=5, bilinear=False)

fig, axes = plt.subplots(1, 2, figsize=(8, 3))
for ax, img, title in zip(axes, [rp_keep, rp_stretch], ['keep_aspect=True', 'keep_aspect=False']):
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Эталон resize/pad через OpenCV: nearest + вручную паддинг

def resize_pad_cv(img, out_h, out_w, keep_aspect=True, pad=0):
    h, w, c = img.shape
    canvas = np.full((out_h, out_w, c), pad, dtype=np.uint8)
    if keep_aspect:
        s = min(out_h / h, out_w / w)
        tgt_h = max(1, int(round(h * s)))
        tgt_w = max(1, int(round(w * s)))
        offy = (out_h - tgt_h) // 2
        offx = (out_w - tgt_w) // 2
    else:
        tgt_h, tgt_w = out_h, out_w
        offy = offx = 0
    resized = cv2.resize(img, (tgt_w, tgt_h), interpolation=cv2.INTER_NEAREST)
    canvas[offy:offy+tgt_h, offx:offx+tgt_w, :] = resized
    return canvas

ref_rp_keep = resize_pad_cv(rgb, 5, 8, keep_aspect=True, pad=5)
ref_rp_stretch = resize_pad_cv(rgb, 5, 8, keep_aspect=False, pad=5)
print("Макс. отличие keep_aspect:", np.abs(ref_rp_keep.astype(int) - rp_keep.astype(int)).max())
print("Макс. отличие stretch:", np.abs(ref_rp_stretch.astype(int) - rp_stretch.astype(int)).max())

## Пайплайн из нескольких операций
Применим цепочку: brightness -> contrast -> add_noise -> resize_pad.

In [ ]:
imgs = [rgb, np.roll(rgb, 1, axis=0)]
processed = []
for im in imgs:
    im1 = px.brightness(im, 20)
    im2 = px.contrast(im1, 1.2)
    im3 = px.add_noise(im2, amp=10, seed=99)
    im4 = px.resize_pad(im3, out_h=5, out_w=8, keep_aspect=True, pad=0, bilinear=False)
    processed.append(im4)

fig, axes = plt.subplots(len(processed), 2, figsize=(8, 3*len(processed)))
for i, (orig, proc) in enumerate(zip(imgs, processed)):
    axes[i, 0].imshow(orig)
    axes[i, 0].set_title(f'orig #{i}')
    axes[i, 0].axis('off')
    axes[i, 1].imshow(proc)
    axes[i, 1].set_title(f'pipeline #{i}')
    axes[i, 1].axis('off')
plt.tight_layout()
plt.show()